In [1]:

from hidromet.utils import dir_projeto
from hidromet import modelos

import geopandas as gpd
from pathlib import Path
import pandas as pd
from pandas.errors import EmptyDataError 
import json

In [2]:
diretorio = dir_projeto()

dir_prec_ana = diretorio.joinpath("arquivos", "series-chuva-ana")
dir_prec_ana.mkdir(parents=True, exist_ok=True)

dir_prec_inmet = diretorio.joinpath("arquivos", "series-chuva-inmet")
dir_prec_inmet.mkdir(parents=True, exist_ok=True)

dir_prec = diretorio.joinpath("arquivos", "series-concatenadas")
dir_prec.mkdir(parents=True, exist_ok=True)

In [10]:
contornos = diretorio.joinpath("contornos", "iguacu.shp")

gdf = gpd.read_file(contornos)
crs = gdf.crs
bacias = gdf.bacia.tolist()

In [13]:
for bacia in bacias:
    try:

        arquivo_serie_ana = dir_prec_ana.joinpath(f"{bacia}.csv")
        arquivo_serie_inmet = dir_prec_inmet.joinpath(f"{bacia}.csv")

        series_bacia_ana = pd.read_csv(arquivo_serie_ana, index_col=0)
        series_bacia_inmet = pd.read_csv(arquivo_serie_inmet, index_col=0)

        arquivo_inmet = dir_prec_inmet.joinpath(f"coords_{bacia}.json")
        dados_inmet = open(arquivo_inmet)
        json_bacia_inmet = json.load(dados_inmet)
        postos_ok_inmet = json_bacia_inmet['postos_ok']
        df_inmet = pd.DataFrame.from_dict(postos_ok_inmet)
        df_inmet = df_inmet.round(2)
        
        arquivo_ana = dir_prec_ana.joinpath(f"coords_{bacia}.json")
        dados_ana = open(arquivo_ana)
        json_bacia_ana = json.load(dados_ana)
        postos_ok_ana = json_bacia_ana['postos_ok']
        df_ana = pd.DataFrame.from_dict(postos_ok_ana)
        df_ana = df_ana.round(2)

        postos_ana = df_ana.codigo
        postos_inmet = df_inmet.codigo
        
        serie_ana = series_bacia_ana[postos_ana]
        serie_inmet = series_bacia_inmet[postos_inmet]
    
        postos_duplicados = df_ana.merge(df_inmet, left_on=["latitude", "longitude"], right_on=["latitude", "longitude"], suffixes=("_ana", "_inmet"))

        if not postos_duplicados.empty:

            for ix in postos_duplicados.index:

                posto_duplicados_ana = postos_duplicados.loc[ix, "codigo_ana"]
                posto_duplicados_inmet = postos_duplicados.loc[ix, "codigo_inmet"]

                serie_duplicada_ana = series_bacia_ana[posto_duplicados_ana].dropna()
                serie_duplicada_inmet = series_bacia_inmet[posto_duplicados_inmet].dropna()

                if len(serie_duplicada_ana) > len(posto_duplicados_inmet):
                    serie_inmet.drop([posto_duplicados_inmet], axis=1)
                    df_inmet = df_inmet[df_inmet.codigo != posto_duplicados_inmet]
                else:
                    serie_ana.drop([posto_duplicados_ana], axis=1)
                    df_ana = df_ana[df_ana.codigo != posto_duplicados_ana]

        log_ana = [modelos.Posto(**df_ana.loc[ix]).dict() for ix in df_ana.index]
        log_inmet = [modelos.Posto(**df_inmet.loc[ix]).dict() for ix in df_inmet.index]

        logs = [*log_ana, *log_inmet]
        arquivo_json = Path(dir_prec, f'coords_{bacia}.json')
        with open(arquivo_json, 'w') as f:
            json.dump(logs, f)

        df = pd.concat([serie_ana, serie_inmet], axis=1)
        df.to_csv(f"{dir_prec}/{bacia}.csv")

    except (EmptyDataError):
        print(f"Não existem séries disponíveis na bacia {bacia}")